In [255]:
import pandas as pd

In [905]:
df_08 = pd.read_csv('data_08_v2.csv')
df_18 = pd.read_csv('data_18_v2.csv')

In [906]:
df_08.dtypes

model                    object
displ                   float64
cyl                      object
trans                    object
drive                    object
fuel                     object
veh_class                object
air_pollution_score      object
city_mpg                 object
hwy_mpg                  object
cmb_mpg                  object
greenhouse_gas_score     object
smartway                 object
dtype: object

In [907]:
df_18.dtypes

model                    object
displ                   float64
cyl                     float64
trans                    object
drive                    object
fuel                     object
veh_class                object
air_pollution_score       int64
city_mpg                 object
hwy_mpg                  object
cmb_mpg                  object
greenhouse_gas_score      int64
smartway                 object
dtype: object

In [908]:
# Extract int from strings in the 2008 cyl column

#https://stackoverflow.com/questions/35376387/extract-int-from-string-in-pandas

# + is 1 or more digits
# \d is a digit (a character in the range 0-9)
# \d+ means match 1 or more digits

In [909]:
## Check value counts for 2008 cyl column again to confirm the change
df_08['cyl'].value_counts()

(6 cyl)     646
(4 cyl)     455
(8 cyl)     392
(5 cyl)     109
(12 cyl)     30
(10 cyl)     15
(2 cyl)       2
(16 cyl)      1
Name: cyl, dtype: int64

In [910]:
df_08['cyl'] = df_08['cyl'].str.extract('(\d+)').astype(int)

In [911]:
# convert 2018 cyl column to int
df_18["cyl"] = df_18.cyl.astype(int)

In [912]:
df_08.to_csv('data_08_v3.csv', index=False)
df_18.to_csv('data_18_v3.csv', index=False)

In [913]:
df_08 = pd.read_csv('data_08_v3.csv')
df_18 = pd.read_csv('data_18_v3.csv')

In [914]:
df_08.shape

(1650, 13)

In [915]:
df_18.shape

(1361, 13)

Fixing air_pollution_score Data Type

2008: convert string to float
2018: convert int to float

In [916]:
# try using pandas' to_numeric or astype function to convert the
# 2008 air_pollution_score column to float -- this won't work

#df_08["air_pollution_score"] = df_08.air_pollution_score.astype(float)

Figuring out the issue
Looks like this isn't going to be as simple as converting the datatype. According to the error above, the air pollution score value in one of the rows is "6/4" - let's check it out. 

df_08[df_08.air_pollution_score == '6/4']
It's not just the air pollution score!
The mpg columns and greenhouse gas scores also seem to have the same problem - maybe that's why these were all saved as strings! According to this link, which I found from the PDF documentation:https://www.fueleconomy.gov/feg/findacarhelp.shtml#airPollutionScore

"If a vehicle can operate on more than one type of fuel, an estimate is provided for each fuel type."
Ohh... so all vehicles with more than one fuel type, or hybrids, like the one above (it uses ethanol AND gas) will have a string that holds two values - one for each. This is a little tricky, so I'm going to show you how to do it with the 2008 dataset, and then you'll try it with the 2018 dataset.

In [917]:
df_08[df_08.air_pollution_score == '6/4']

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway
1022,MERCEDES-BENZ C300,3.0,6,Auto-L7,2WD,ethanol/gas,small car,6/4,13/18,19/25,15/21,7/6,no


In [918]:
# First, let's get all the hybrids in 2008
hb_08 = df_08[df_08['fuel'].str.contains('/')]
hb_08

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway
209,CHEVROLET Avalanche 1500,5.3,8,Auto-L4,2WD,ethanol/gas,SUV,7/7,11/14,15/20,12/16,6/4,no
210,CHEVROLET Avalanche 1500,5.3,8,Auto-L4,4WD,ethanol/gas,SUV,7/7,11/14,14/19,12/16,6/4,no
214,CHEVROLET Avalanche 1500,5.3,8,Auto-L4,2WD,ethanol/gas,SUV,6/6,11/14,15/20,12/16,6/4,no
215,CHEVROLET Avalanche 1500,5.3,8,Auto-L4,4WD,ethanol/gas,SUV,6/6,11/14,14/19,12/16,6/4,no
262,CHEVROLET Express 1500,5.3,8,Auto-L4,4WD,ethanol/gas,van,6/6,9/12,12/16,10/14,4/2,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1149,MITSUBISHI Raider,4.7,8,Auto-L5,4WD,ethanol/gas,pickup,3/3,9/14,12/19,10/15,4/3,no
1175,NISSAN Armada,5.6,8,Auto-L5,2WD,ethanol/gas,SUV,6/6,9/12,13/18,10/14,4/2,no
1176,NISSAN Armada,5.6,8,Auto-L5,4WD,ethanol/gas,SUV,6/6,9/12,13/17,10/14,4/2,no
1211,NISSAN Titan,5.6,8,Auto-L5,2WD,ethanol/gas,pickup,6/6,9/12,13/17,10/14,4/2,no


Looks like this dataset only has one! The 2018 has MANY more - but don't worry - the steps I'm taking here will work for that as well!

In [919]:
# hybrids in 2018
hb_18 = df_18[df_18['fuel'].str.contains('/')]
hb_18

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway
30,AUDI A3 e-tron,1.4,4,AMS-6,2WD,Gasoline/Electricity,small car,7,34/81,39/86,36/83,10,Elite
88,BMW 330e,2.0,4,SemiAuto-8,2WD,Gasoline/Electricity,small car,3,28/66,34/78,30/71,10,Yes
114,BMW 530e,2.0,4,SemiAuto-8,2WD,Gasoline/Electricity,small car,7,27/70,31/75,29/72,10,Elite
115,BMW 530e,2.0,4,SemiAuto-8,4WD,Gasoline/Electricity,small car,7,27/66,31/68,28/67,10,Elite
130,BMW 740e,2.0,4,SemiAuto-8,4WD,Gasoline/Electricity,large car,3,25/62,29/68,27/64,9,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1285,TOYOTA Sequoia FFV,5.7,8,SemiAuto-6,4WD,Ethanol/Gas,standard SUV,5,9/13,13/17,10/14,1,No
1298,TOYOTA Tundra FFV,5.7,8,SemiAuto-6,4WD,Ethanol/Gas,pickup,5,9/13,12/17,10/15,2,No
1344,VOLVO S90,2.0,4,SemiAuto-8,4WD,Gasoline/Electricity,midsize car,7,26/70,33/72,29/71,10,Elite
1356,VOLVO XC60,2.0,4,SemiAuto-8,4WD,Gasoline/Electricity,small SUV,7,26/60,28/58,26/59,10,Elite


We're going to take each hybrid row and split them into two new rows - one with values for the first fuel type (values before the "/"), and the other with values for the second fuel type (values after the "/"). Let's separate them with two dataframes!

In [920]:
# create two copies of the 2008 hybrids dataframe
df1 = hb_08.copy()  # data on first fuel type of each hybrid vehicle
df2 = hb_08.copy()  # data on second fuel type of each hybrid vehicle

# Each one should look like this
#df1

For this next part, we're going use pandas' apply function. See the docs https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html

In [921]:
# columns to split by "/"
split_columns = ['fuel', 'air_pollution_score', 'city_mpg', 'hwy_mpg', 'cmb_mpg', 'greenhouse_gas_score']


In [922]:
# apply split function to each column of each dataframe copy

for c in split_columns:
    df1[c] = df1[c].apply(lambda x: x.split("/")[0])
    df2[c] = df2[c].apply(lambda x: x.split("/")[1])

In [923]:
# this dataframe holds info for the FIRST fuel type of the hybrid
# aka the values before the "/"s

df1

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway
209,CHEVROLET Avalanche 1500,5.3,8,Auto-L4,2WD,ethanol,SUV,7,11,15,12,6,no
210,CHEVROLET Avalanche 1500,5.3,8,Auto-L4,4WD,ethanol,SUV,7,11,14,12,6,no
214,CHEVROLET Avalanche 1500,5.3,8,Auto-L4,2WD,ethanol,SUV,6,11,15,12,6,no
215,CHEVROLET Avalanche 1500,5.3,8,Auto-L4,4WD,ethanol,SUV,6,11,14,12,6,no
262,CHEVROLET Express 1500,5.3,8,Auto-L4,4WD,ethanol,van,6,9,12,10,4,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1149,MITSUBISHI Raider,4.7,8,Auto-L5,4WD,ethanol,pickup,3,9,12,10,4,no
1175,NISSAN Armada,5.6,8,Auto-L5,2WD,ethanol,SUV,6,9,13,10,4,no
1176,NISSAN Armada,5.6,8,Auto-L5,4WD,ethanol,SUV,6,9,13,10,4,no
1211,NISSAN Titan,5.6,8,Auto-L5,2WD,ethanol,pickup,6,9,13,10,4,no


In [924]:
# this dataframe holds info for the SECOND fuel type of the hybrid
# aka the values after the "/"s

df2

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway
209,CHEVROLET Avalanche 1500,5.3,8,Auto-L4,2WD,gas,SUV,7,14,20,16,4,no
210,CHEVROLET Avalanche 1500,5.3,8,Auto-L4,4WD,gas,SUV,7,14,19,16,4,no
214,CHEVROLET Avalanche 1500,5.3,8,Auto-L4,2WD,gas,SUV,6,14,20,16,4,no
215,CHEVROLET Avalanche 1500,5.3,8,Auto-L4,4WD,gas,SUV,6,14,19,16,4,no
262,CHEVROLET Express 1500,5.3,8,Auto-L4,4WD,gas,van,6,12,16,14,2,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1149,MITSUBISHI Raider,4.7,8,Auto-L5,4WD,gas,pickup,3,14,19,15,3,no
1175,NISSAN Armada,5.6,8,Auto-L5,2WD,gas,SUV,6,12,18,14,2,no
1176,NISSAN Armada,5.6,8,Auto-L5,4WD,gas,SUV,6,12,17,14,2,no
1211,NISSAN Titan,5.6,8,Auto-L5,2WD,gas,pickup,6,12,17,14,2,no


In [925]:
# combine dataframes to add to the original dataframe
new_df = df1.append(df2)

# now we have separate rows for each fuel type of each vehicle!
new_df

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway
209,CHEVROLET Avalanche 1500,5.3,8,Auto-L4,2WD,ethanol,SUV,7,11,15,12,6,no
210,CHEVROLET Avalanche 1500,5.3,8,Auto-L4,4WD,ethanol,SUV,7,11,14,12,6,no
214,CHEVROLET Avalanche 1500,5.3,8,Auto-L4,2WD,ethanol,SUV,6,11,15,12,6,no
215,CHEVROLET Avalanche 1500,5.3,8,Auto-L4,4WD,ethanol,SUV,6,11,14,12,6,no
262,CHEVROLET Express 1500,5.3,8,Auto-L4,4WD,ethanol,van,6,9,12,10,4,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1149,MITSUBISHI Raider,4.7,8,Auto-L5,4WD,gas,pickup,3,14,19,15,3,no
1175,NISSAN Armada,5.6,8,Auto-L5,2WD,gas,SUV,6,12,18,14,2,no
1176,NISSAN Armada,5.6,8,Auto-L5,4WD,gas,SUV,6,12,17,14,2,no
1211,NISSAN Titan,5.6,8,Auto-L5,2WD,gas,pickup,6,12,17,14,2,no


In [926]:
# add in our newly separated rows
df_08 = df_08.append(new_df, ignore_index=True)

In [927]:
df_08

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway
0,ACURA MDX,3.7,6,Auto-S5,4WD,Gasoline,SUV,7,15,20,17,4,no
1,ACURA MDX,3.7,6,Auto-S5,4WD,Gasoline,SUV,6,15,20,17,4,no
2,ACURA RDX,2.3,4,Auto-S5,4WD,Gasoline,SUV,7,17,22,19,5,no
3,ACURA RDX,2.3,4,Auto-S5,4WD,Gasoline,SUV,6,17,22,19,5,no
4,ACURA RL,3.5,6,Auto-S5,4WD,Gasoline,midsize car,7,16,24,19,5,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1775,MITSUBISHI Raider,4.7,8,Auto-L5,4WD,gas,pickup,3,14,19,15,3,no
1776,NISSAN Armada,5.6,8,Auto-L5,2WD,gas,SUV,6,12,18,14,2,no
1777,NISSAN Armada,5.6,8,Auto-L5,4WD,gas,SUV,6,12,17,14,2,no
1778,NISSAN Titan,5.6,8,Auto-L5,2WD,gas,pickup,6,12,17,14,2,no


In [928]:
df_08.tail(3)

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway
1777,NISSAN Armada,5.6,8,Auto-L5,4WD,gas,SUV,6,12,17,14,2,no
1778,NISSAN Titan,5.6,8,Auto-L5,2WD,gas,pickup,6,12,17,14,2,no
1779,NISSAN Titan,5.6,8,Auto-L5,4WD,gas,pickup,6,12,17,14,2,no


In [929]:
# drop the original hybrid rows
df_08.drop(hb_08.index, inplace=True)

In [930]:
# check that all the original hybrid rows with "/"s are gone
df_08[df_08['fuel'].str.contains('/')]

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway


In [931]:
# hybrids in 2018
hb_18 = df_18[df_18['fuel'].str.contains('/')]
hb_18

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway
30,AUDI A3 e-tron,1.4,4,AMS-6,2WD,Gasoline/Electricity,small car,7,34/81,39/86,36/83,10,Elite
88,BMW 330e,2.0,4,SemiAuto-8,2WD,Gasoline/Electricity,small car,3,28/66,34/78,30/71,10,Yes
114,BMW 530e,2.0,4,SemiAuto-8,2WD,Gasoline/Electricity,small car,7,27/70,31/75,29/72,10,Elite
115,BMW 530e,2.0,4,SemiAuto-8,4WD,Gasoline/Electricity,small car,7,27/66,31/68,28/67,10,Elite
130,BMW 740e,2.0,4,SemiAuto-8,4WD,Gasoline/Electricity,large car,3,25/62,29/68,27/64,9,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1285,TOYOTA Sequoia FFV,5.7,8,SemiAuto-6,4WD,Ethanol/Gas,standard SUV,5,9/13,13/17,10/14,1,No
1298,TOYOTA Tundra FFV,5.7,8,SemiAuto-6,4WD,Ethanol/Gas,pickup,5,9/13,12/17,10/15,2,No
1344,VOLVO S90,2.0,4,SemiAuto-8,4WD,Gasoline/Electricity,midsize car,7,26/70,33/72,29/71,10,Elite
1356,VOLVO XC60,2.0,4,SemiAuto-8,4WD,Gasoline/Electricity,small SUV,7,26/60,28/58,26/59,10,Elite


In [932]:
# create two copies of the 2018 hybrids dataframe
df3 = hb_18.copy()  # data on first fuel type of each hybrid vehicle
df4 = hb_18.copy()  # data on second fuel type of each hybrid vehicle

# Each one should look like this
#df3

In [933]:
# columns to split by "/"
#Split values for fuel, city_mpg, hwy_mpg, cmb_mpg
#You don't need to split for air_pollution_score or greenhouse_gas_score here because these columns are already ints in the 2018 dataset.

split_columns = ['fuel', 'city_mpg', 'hwy_mpg', 'cmb_mpg']


In [934]:
df3.dtypes

model                    object
displ                   float64
cyl                       int64
trans                    object
drive                    object
fuel                     object
veh_class                object
air_pollution_score       int64
city_mpg                 object
hwy_mpg                  object
cmb_mpg                  object
greenhouse_gas_score      int64
smartway                 object
dtype: object

In [935]:
# apply split function to each column of each dataframe copy

for c in split_columns:
    df3[c] = df3[c].apply(lambda x: x.split("/")[0])
    df4[c] = df4[c].apply(lambda x: x.split("/")[1])

In [936]:
# combine dataframes to add to the original dataframe
new_df_18 = df3.append(df4)

# now we have separate rows for each fuel type of each vehicle!
new_df_18

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway
30,AUDI A3 e-tron,1.4,4,AMS-6,2WD,Gasoline,small car,7,34,39,36,10,Elite
88,BMW 330e,2.0,4,SemiAuto-8,2WD,Gasoline,small car,3,28,34,30,10,Yes
114,BMW 530e,2.0,4,SemiAuto-8,2WD,Gasoline,small car,7,27,31,29,10,Elite
115,BMW 530e,2.0,4,SemiAuto-8,4WD,Gasoline,small car,7,27,31,28,10,Elite
130,BMW 740e,2.0,4,SemiAuto-8,4WD,Gasoline,large car,3,25,29,27,9,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1285,TOYOTA Sequoia FFV,5.7,8,SemiAuto-6,4WD,Gas,standard SUV,5,13,17,14,1,No
1298,TOYOTA Tundra FFV,5.7,8,SemiAuto-6,4WD,Gas,pickup,5,13,17,15,2,No
1344,VOLVO S90,2.0,4,SemiAuto-8,4WD,Electricity,midsize car,7,70,72,71,10,Elite
1356,VOLVO XC60,2.0,4,SemiAuto-8,4WD,Electricity,small SUV,7,60,58,59,10,Elite


In [937]:
# add in our newly separated rows
df_18 = df_18.append(new_df_18, ignore_index=True)

In [938]:
# drop the original hybrid rows
df_18.drop(hb_18.index, inplace=True)

In [939]:
# check that all the original hybrid rows with "/"s are gone
df_18[df_18['fuel'].str.contains('/')]

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway


Now we can comfortably continue the changes needed for air_pollution_score! Here they are again:
2008: convert string to float
2018: convert int to float

In [940]:
def change_type_from_string(df, col_name, dtype):
    return df[col_name].str.extract('(\d+)').astype(dtype)

In [941]:
def change_type(df, col_name, dtype):
    return df[col_name].astype(dtype)

In [942]:
df_08["air_pollution_score"] = change_type_from_string(df_08,"air_pollution_score",float)

In [943]:
df_18["air_pollution_score"] = change_type(df_18,"air_pollution_score",float)

In [944]:
#Fix city_mpg, hwy_mpg, cmb_mpg datatypes

#2008 and 2018: convert string to float

df_08["city_mpg"] = change_type_from_string(df_08,"city_mpg",float)

In [945]:
df_18["city_mpg"] = change_type_from_string(df_18,"city_mpg",float)

In [946]:
df_08["hwy_mpg"] = change_type_from_string(df_08,"hwy_mpg",float)

In [947]:
df_18["hwy_mpg"] = change_type_from_string(df_18,"hwy_mpg",float)

In [948]:
df_08["cmb_mpg"] = change_type_from_string(df_08,"cmb_mpg",float)

In [949]:
df_18["cmb_mpg"] = change_type_from_string(df_18,"cmb_mpg",float)

In [950]:
#Fix greenhouse_gas_score datatype

df_08["greenhouse_gas_score"] = change_type_from_string(df_08,"greenhouse_gas_score",int)

In [951]:
df_08.shape

(1715, 13)

In [952]:
df_18.shape

(1450, 13)

In [953]:
df_08.to_csv('data_08_v4.csv', index=False)
df_18.to_csv('data_18_v4.csv', index=False)

In [954]:
# load datasets

df_08 = pd.read_csv('data_08_v4.csv')
df_18 = pd.read_csv('data_18_v4.csv')